## Preprocessing

In [57]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("../Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


### Note:
#### Features:
APPLICATION_TYPE 	AFFILIATION 	CLASSIFICATION 	USE_CASE 	ORGANIZATION 	STATUS 	INCOME_AMT 	SPECIAL_CONSIDERATIONS 	ASK_AMT

#### Target: 
IS_SUCCESSFUL

In [58]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN','NAME'])

In [59]:
# Determine the number of unique values in each column.
for c in application_df.columns:
    print(f'{c:<30}{len(application_df[c].unique())}')

APPLICATION_TYPE              17
AFFILIATION                   6
CLASSIFICATION                71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                       8747
IS_SUCCESSFUL                 2


In [60]:
# Look at APPLICATION_TYPE value counts for binning
application_df.APPLICATION_TYPE.value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [61]:
# Look at <BLANK> value counts for binning
application_df.ASK_AMT.value_counts()

5000        25398
10478           3
15583           3
63981           3
6725            3
            ...  
5371754         1
30060           1
43091152        1
18683           1
36500179        1
Name: ASK_AMT, Length: 8747, dtype: int64

In [73]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1003
C7000      777
C1700      287
C4000      194
Name: CLASSIFICATION, dtype: int64

In [62]:
# This cell generalizes replacement cutoffs for any column listed, as well as a corresponding cutoff level.
# To use elsewhere, check the column names within the dataframe.
# CHECK DATAFRAME NAME

column_list = ['APPLICATION_TYPE', 'CLASSIFICATION']
cutoff_list = [200               , 150             ]
for i in range(len(column_list)):
    column = column_list[i]
    cutoff = cutoff_list[i]
    # list of unique values within single column
                    ########## DATAFRAME
    col_type_list = application_df[column].unique().tolist()
    col_type_list
#     print(len(col_type_list))
    
    # list of counts of each unique value within single column
    col_amt_list = []
    for col in col_type_list:
                                ########## DATAFRAME       ########## DATAFRAME
        col_amt_list.append(len(application_df[column].loc[application_df[column]==col]))
    col_amt_list
#     print(len(col_amt_list))

    reduced_col_list = []
    for i in range(len(col_type_list)):
        if col_amt_list[i] <= cutoff:
            reduced_col_list.append(col_type_list[i])
    reduced_col_list

    
    for name in reduced_col_list:
        ########## DATAFRAME     ########## DATAFRAME
        application_df[column] = application_df[column].replace({
            name: 'Other'
        })
    print(column)
          ########## DATAFRAME
    print(application_df[column].value_counts())
    print('#'*30)

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64
##############################
CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1003
C7000      777
C1700      287
C4000      194
Name: CLASSIFICATION, dtype: int64
##############################


In [74]:
# Convert categorical data to numeric with `pd.get_dummies`
application_dummies = pd.get_dummies(application_df)
application_dummies

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [75]:
# Split our preprocessed data into our features and target arrays
X_dummies = application_dummies.drop(columns=['IS_SUCCESSFUL'])
y = application_dummies['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X_dummies, y, random_state=1)

In [78]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

X_train_scaled.shape

(25724, 46)

## Compile, Train and Evaluate the Model

In [84]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=50, activation='relu', input_dim = 46))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation="tanh"))

# Output layer

nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 50)                2350      
                                                                 
 dense_5 (Dense)             (None, 30)                1530      
                                                                 
 dense_6 (Dense)             (None, 1)                 31        
                                                                 
Total params: 3,911
Trainable params: 3,911
Non-trainable params: 0
_________________________________________________________________


In [85]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [90]:
# Weight Saver Callback Function
from keras.callbacks import Callback
class WeightsSaver(Callback):
  def __init__(self, N):
    self.N = N
    self.epoch = 0

  def on_epoch_end(self, epoch, logs={}):
    if self.epoch % self.N == 0:
      name = 'weights%08d.h5' % self.epoch
      folder = '../Models/Weights/'
      self.model.save_weights(folder + name)
    self.epoch += 1


# Train the model
fit_model = nn.fit(X_train_scaled,y_train, epochs=150, callbacks=[WeightsSaver(5)])

Epoch 1/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5317 - accuracy: 0.7403
Epoch 2/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5316 - accuracy: 0.7408
Epoch 3/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5317 - accuracy: 0.7415
Epoch 4/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5315 - accuracy: 0.7418
Epoch 5/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5317 - accuracy: 0.7416
Epoch 6/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5313 - accuracy: 0.7417
Epoch 7/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5317 - accuracy: 0.7415
Epoch 8/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5315 - accuracy: 0.7414
Epoch 9/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5316 - accuracy: 0.7412
Epoch 10/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5318 - accura

804/804 [==============================] - 1s 1ms/step - loss: 0.5311 - accuracy: 0.7418
Epoch 82/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5310 - accuracy: 0.7413
Epoch 83/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5314 - accuracy: 0.7407
Epoch 84/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5311 - accuracy: 0.7415
Epoch 85/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5308 - accuracy: 0.7420
Epoch 86/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5310 - accuracy: 0.7408
Epoch 87/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5306 - accuracy: 0.7412
Epoch 88/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5306 - accuracy: 0.7416
Epoch 89/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5310 - accuracy: 0.7407
Epoch 90/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5314 - accuracy: 

In [91]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5576 - accuracy: 0.7294 - 297ms/epoch - 1ms/step
Loss: 0.5576171278953552, Accuracy: 0.7294460535049438


In [92]:
# Export our model to HDF5 file
nn.save("../ModelsAlphabetSoupCharity.h5")